In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rc
import matplotlib.pyplot as plt


rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus']=False  

# 전체 데이터

In [13]:
total = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/total_cust.csv', index_col = 0)
total.head()

,1분기 대비2분기 증감여부,2분기 대비3분기 증감여부,3분기 대비4분기 증감여부,4분기 대비5분기 증감여부,5분기 대비6분기 증감여부,6분기 대비7분기 증감여부,7분기 대비8분기 증감여부
CUSTNO,,,,,,,
1,3,3,3,1,2,3,1
2,2,4,1,4,1,4,2
3,4,3,2,2,3,3,2
4,4,2,1,3,3,3,2
6,3,3,2,3,2,3,2


In [48]:
# 1분기
year2014_q1 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2014_q1.csv', index_col = 0)
year2014_q1['purchase_sum'] = year2014_q1.apply(lambda x:x.sum(), axis = 1)

# 2분기
year2014_q2 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2014_q2.csv', index_col = 0)
year2014_q2['purchase_sum'] = year2014_q2.apply(lambda x:x.sum(), axis = 1)

# 3분기
year2014_q3 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2014_q3.csv', index_col = 0)
year2014_q3['purchase_sum'] = year2014_q3.apply(lambda x:x.sum(), axis = 1)

#4분기
year2014_q4 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2014_q4.csv', index_col = 0)
year2014_q4['purchase_sum'] = year2014_q4.apply(lambda x:x.sum(), axis = 1)

#5분기
year2015_q1 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2015_q1.csv', index_col = 0)
year2015_q1['purchase_sum'] = year2015_q1.apply(lambda x:x.sum(), axis = 1)

# 6분기
year2015_q2 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2015_q2.csv', index_col = 0)
year2015_q2['purchase_sum'] = year2015_q2.apply(lambda x:x.sum(), axis = 1)

#7분기
year2015_q3 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2015_q3.csv', index_col = 0)
year2015_q3['purchase_sum'] = year2015_q3.apply(lambda x:x.sum(), axis = 1)

#8분기
year2015_q4 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weight_plus_2015_q4.csv', index_col = 0)
year2015_q4['purchase_sum'] = year2015_q4.apply(lambda x:x.sum(), axis = 1)

# 데이터 리스트
df_li = [year2014_q1, year2014_q2, year2014_q3, year2014_q4, year2015_q1, year2015_q2, year2015_q3, year2015_q4]
total_columns = year2015_q3.columns.tolist()

plus_or_minus_cust = []
for i in total_columns:
    feature = i + '_증감여부'
    plus_or_minus_cust.append(feature)
plus_or_minus_cust

def quater_weight_df(x, y): #1분기 대비 2분기 or 1분기 대비 3분기 각 분기 대입
    # 기준 원본 데이터
    df_q1 = df_li[x-1]
    # 비교할 원본 데이터
    df_q2 = df_li[y-1]
    
    # 대입할 데이터프레임 생성
    total_df = pd.DataFrame(columns = plus_or_minus_cust, index = year2015_q3.index)
    
    # 각 컬럼명 
    for i in total_columns:
        # 각 컬럼별 평균 증감율 = (비교할 데이터 - 기준 원본 데이터)/기준 원본 데이터
        mean_ratio = (df_q2.loc[:, i].sum() - df_q1.loc[:, i].sum())/df_q1.loc[:, i].sum()
        
        # 기준 금액 : 기준 원본데이터 * (1+mean_ratio)
        stand_price = df_q1.loc[:, i] * (1+mean_ratio)
        # 기준금액 컬럼명
        stand_column = i + '_기준금액'


        # 기준 금액 비교할 데이터프레임에 추가
        df_q2[stand_column] = stand_price

        # 비교할 원본 데이터 금액 과 기준금액 비교 후 크면 2, 작으면 0, 같으면 1 대입 
        # 비교할 원본 금액과 기준 금액 컬럼만 따로 추출
        df_whos_ = df_q2[[i, stand_column]]

        # 증가 고객
        df_whos_plus = df_whos_[df_whos_[i] > df_whos_[stand_column]] # 기준금액 보다 큰 경우
        # 감소 고객
        df_whos_minus = df_whos_[df_whos_[i] <= df_whos_[stand_column]] # 기준금액 보다 작은 경우

#         # 유지 고객
#         df_whos_same = df_whos_[df_whos_[i] == df_whos_[stand_column]] # 기준금액이랑 차이 없음

        # total_df에 값 대입
        input_column_name = i + '_증감여부'
        # 증가면 0
        total_df.loc[df_whos_plus.index, input_column_name] = 0
        # 감소면 1
        total_df.loc[df_whos_minus.index, input_column_name] = 1

    return total_df['purchase_sum_증감여부']

## RFM 파일

In [27]:
rfm2 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/RFM_q2.csv', index_col = 0)
rfm2.head()

,D-Recency,Frequancy,Monetary,Recency
CUSTNO,,,,
1,1,238,19925355,20140630
2,1,339,24323230,20140630
3,1,348,851215,20140630
4,2,270,5042106,20140629
6,1,384,5281290,20140630


In [28]:
rfm3 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/RFM_q3.csv', index_col = 0)
rfm3.head()

,D-Recency,Frequancy,Monetary,Recency
CUSTNO,,,,
1,5,259,21565315,20140926
2,4,294,25040940,20140927
3,1,455,1196621,20140930
4,1,262,6166873,20140930
6,2,370,6278557,20140929


In [29]:
rfm4 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/RFM_q4.csv', index_col = 0)
rfm4.head()

,D-Recency,Frequancy,Monetary,Recency
CUSTNO,,,,
1,1,356,34116312,20141231
2,1,271,21922100,20141231
3,2,391,1042554,20141230
4,1,172,4257510,20141231
6,2,365,6114089,20141230


In [30]:
rfm5 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/RFM_q5.csv', index_col = 0)
rfm5.head()

,D-Recency,Frequancy,Monetary,Recency
CUSTNO,,,,
1,3,386,32637747,20150329
2,1,277,25562400,20150331
3,1,232,673676,20150331
4,2,209,3560392,20150330
6,1,424,6570050,20150331


In [31]:
rfm6 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/RFM_q6.csv', index_col = 0)
rfm6.head()

,D-Recency,Frequancy,Monetary,Recency
CUSTNO,,,,
1,4,312,17898365,20150627
2,7,261,27791610,20150624
3,2,220,592280,20150629
4,1,263,3656937,20150630
6,1,397,6389055,20150630


In [32]:
rfm7 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/RFM_q7.csv', index_col = 0)
rfm7.head()

,D-Recency,Frequancy,Monetary,Recency
CUSTNO,,,,
1,9,213,16401960,20150922
2,2,242,23152730,20150929
3,9,272,733987,20150922
4,1,259,4046757,20150930
6,1,388,6287915,20150930


# 1. 1-2분기 Target : 2-3분기

In [11]:
df1 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q2_q1_df.csv', index_col = 0)
df1.head()

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,4,1,2,0,3,1,0,1
2,0,2,3,2,1,4,0,0,4,3,0,2,2,0,4,2,0,2
3,0,3,0,0,0,0,0,0,0,3,3,4,0,0,4,0,0,0
4,0,2,3,4,4,2,0,0,0,4,4,4,4,2,4,0,4,1
6,4,4,4,0,4,0,0,0,0,2,1,3,2,3,4,4,2,1


In [14]:
df1['total증감여부'] = total['1분기 대비2분기 증감여부']
df1.head()

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부
CUSTNO,,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,4,1,2,0,3,1,0,1,3
2,0,2,3,2,1,4,0,0,4,3,0,2,2,0,4,2,0,2,2
3,0,3,0,0,0,0,0,0,0,3,3,4,0,0,4,0,0,0,4
4,0,2,3,4,4,2,0,0,0,4,4,4,4,2,4,0,4,1,4
6,4,4,4,0,4,0,0,0,0,2,1,3,2,3,4,4,2,1,3


In [19]:
df1['label'] = quater_weight_df(2,3)
df1

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부,label
CUSTNO,,,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,4,1,2,0,3,1,0,1,3,0
2,0,2,3,2,1,4,0,0,4,3,0,2,2,0,4,2,0,2,2,0
3,0,3,0,0,0,0,0,0,0,3,3,4,0,0,4,0,0,0,4,0
4,0,2,3,4,4,2,0,0,0,4,4,4,4,2,4,0,4,1,4,1
6,4,4,4,0,4,0,0,0,0,2,1,3,2,3,4,4,2,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,4,0,0,1,0,0,1,3,4,2,4,0,0,1,0,1,0,3,1
19334,0,0,0,0,4,0,0,4,4,4,4,4,0,2,0,0,4,0,4,1
19335,0,3,0,0,4,4,3,4,3,4,4,4,0,4,4,0,2,4,4,1


In [34]:
rfm2_ = rfm2.drop('Recency', axis=1, inplace=False)
total_2 = pd.concat([df1, rfm2_], axis=1)
total_2

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,...,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부,label,D-Recency,Frequancy,Monetary
CUSTNO,,,,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,...,0,3,1,0,1,3,0,1,238,19925355
2,0,2,3,2,1,4,0,0,4,3,...,0,4,2,0,2,2,0,1,339,24323230
3,0,3,0,0,0,0,0,0,0,3,...,0,4,0,0,0,4,0,1,348,851215
4,0,2,3,4,4,2,0,0,0,4,...,2,4,0,4,1,4,1,2,270,5042106
6,4,4,4,0,4,0,0,0,0,2,...,3,4,4,2,1,3,0,1,384,5281290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,4,0,0,1,0,0,1,3,4,...,0,1,0,1,0,3,1,1,281,1527118
19334,0,0,0,0,4,0,0,4,4,4,...,2,0,0,4,0,4,1,1,629,3539966
19335,0,3,0,0,4,4,3,4,3,4,...,4,4,0,2,4,4,1,2,390,3720856


# 모델

In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

def get_clf_eval(y_test,pred,pred_proba):
    accuracy = accuracy_score(y_test,pred)
    print(f'정확도: {accuracy}')
    
def get_model_train_eval(df, models):
    x = df.drop('label', axis = 1)
    y = df['label']

    # 학습/테스트용 분리
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 11)
    
    for model in models:
        print(model)
        model.fit(X_train, y_train)
        pred=model.predict(X_test)
        pred_proba=model.predict_proba(X_test)[:,1]
        get_clf_eval(y_test,pred,pred_proba)


# 모델리스트
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, 
                         boost_from_average=False)
total_2['label']= total_2['label'].astype('int')
model_li = [dt_clf, rf_clf, lgbm_clf]
get_model_train_eval(total_2, model_li)

DecisionTreeClassifier()
정확도: 0.555908850026497
RandomForestClassifier()
정확도: 0.6414944356120826
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.611552729199788


## 다른 분기

In [37]:
# 2분기 3분기
df2 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q3_q2_df.csv', index_col = 0)
# 3분기 4분기
df3 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q4_q3_df.csv', index_col = 0)
# 4분기 5분기
df4 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q5_q4_df.csv', index_col = 0)
# 5분기 6분기
df5= pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q6_q5_df.csv', index_col = 0)
# 6분기 7분기
df6= pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q7_q6_df.csv', index_col = 0)

In [49]:
# 전체 데이터 
totalcolumns = total.columns[1:]

# 분기별 데이터
q_df = [df2, df3, df4, df5, df6]
# rfm 데이터
rfm = [rfm3, rfm4, rfm5, rfm6, rfm7]

for ind, name in enumerate(totalcolumns):
    total_df = q_df[ind]
    total_df['total증감여부'] = total[name]
    before_ = ind+3
    after_ = ind+4
    total_df['label'] = quater_weight_df(before_, after_)
    
    rfm_ = rfm[ind].drop('Recency', axis=1, inplace=False)
    total_2 = pd.concat([total_df, rfm_], axis=1)
    total_2['label'] = total_2['label'].astype('int')
    model_li = [dt_clf, rf_clf, lgbm_clf]
    print(f'---Target : {before_}대비 {after_}---')
    get_model_train_eval(total_2, model_li)

---Target : 3대비 4---
DecisionTreeClassifier()
정확도: 0.5723370429252782
RandomForestClassifier()
정확도: 0.6555378908320085
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.6311605723370429
---Target : 4대비 5---
DecisionTreeClassifier()
정확도: 0.5694223635400106
RandomForestClassifier()
정확도: 0.634870164281929
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.6221515633280339
---Target : 5대비 6---
DecisionTreeClassifier()
정확도: 0.591679915209327
RandomForestClassifier()
정확도: 0.6507684154742979
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.6409644939056703
---Target : 6대비 7---
DecisionTreeClassifier()
정확도: 0.5482246952835188
RandomForestClassifier()
정확도: 0.6165871754107048
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.5969793322734499
---Target : 7대비 8---
DecisionTreeClassifier()
정확도: 0.5569687334393216
RandomForestClassifier()
정확도: 0.6332803391626921
LGBMClassif

IndexError: list index out of range

In [54]:
# 전체 데이터 
totalcolumns = total.columns[1:]

# 분기별 데이터
q_df = [df2, df3, df4, df5, df6]
# rfm 데이터
rfm = [rfm3, rfm4, rfm5, rfm6, rfm7]

def concatQ(q_dfs, rfms, before, after):
    total_df = q_dfs
    total_df['total증감여부'] = total[name]
    total_df['label'] = quater_weight_df(before, after)
    rfm_ = rfms.drop('Recency', axis = 1, inplace = False)
    total_2 = pd.concat([total_df, rfm_], axis=1)
    total_2['label'] = total_2['label'].astype('int')
    return total_2

total_1 = concatQ(df1, rfm2, 2, 3)
total_2 = concatQ(df2, rfm3, 3, 4)
total_3 = concatQ(df3, rfm4, 4, 5)
total_4 = concatQ(df4, rfm5, 5, 6)
total_5 = concatQ(df5, rfm6, 6, 7)
total_6 = concatQ(df6, rfm7, 7, 8)

#     for ind, name in enumerate(totalcolumns):
#     total_df = q_df[ind]
#     total_df['total증감여부'] = total[name]
#     before_ = ind+3
#     after_ = ind+4
#     total_df['label'] = quater_weight_df(before_, after_)
    
#     rfm_ = rfm[ind].drop('Recency', axis=1, inplace=False)
#     total_2 = pd.concat([total_df, rfm_], axis=1)
#     total_2['label'] = total_2['label'].astype('int')
#     model_li = [dt_clf, rf_clf, lgbm_clf]
#     print(f'---Target : {before_}대비 {after_}---')
#     get_model_train_eval(total_2, model_li)

In [55]:
Q_total = pd.concat([total_1, total_2, total_3, total_4,total_5, total_6])
Q_total

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,...,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부,label,D-Recency,Frequancy,Monetary
CUSTNO,,,,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,...,0,3,1,0,1,1,0,1,238,19925355
2,0,2,3,2,1,4,0,0,4,3,...,0,4,2,0,2,2,0,1,339,24323230
3,0,3,0,0,0,0,0,0,0,3,...,0,4,0,0,0,2,0,1,348,851215
4,0,2,3,4,4,2,0,0,0,4,...,2,4,0,4,1,2,1,2,270,5042106
6,4,4,4,0,4,0,0,0,0,2,...,3,4,4,2,1,2,0,1,384,5281290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,4,0,3,124,805600
19334,0,1,0,0,0,0,4,1,4,1,...,1,4,0,1,0,2,1,32,202,1252229
19335,0,2,1,0,3,1,3,2,3,1,...,1,0,0,2,0,3,0,1,444,4406340


# 전체 합친거 모델링

In [56]:
def get_clf_eval(y_test,pred):
    accuracy = accuracy_score(y_test,pred)
    print(f'정확도: {accuracy}')
    
def get_model_train_eval(df, models):
    x = df.drop('label', axis = 1)
    y = df['label']

    # 학습/테스트용 분리
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 11)
    
    for model in models:
        print(model)
        model.fit(X_train, y_train)
        pred=model.predict(X_test)
        get_clf_eval(y_test,pred)

# 모델리스트
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, 
                         boost_from_average=False)
model_li = [dt_clf, rf_clf, lgbm_clf]
get_model_train_eval(Q_total, model_li)

DecisionTreeClassifier()
정확도: 0.5522968197879858
RandomForestClassifier()
정확도: 0.627517667844523
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.6259717314487633


# 주말 비율 추가

In [57]:
weekend_ratio = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/weekend_ratio.csv', index_col = 0)
weekend_ratio.head()

,주말,평일,주말_비율
고객번호,,,
1,227.0,830.0,0.21
2,326.0,790.0,0.29
3,226.0,965.0,0.19
4,214.0,716.0,0.23
6,387.0,1206.0,0.24


In [76]:
weekend_ratio[weekend_ratio['주말_비율'].isnull() == True]

,주말,평일,주말_비율
고객번호,,,
6052,NaN,912.0,NaN
9652,NaN,739.0,NaN


In [62]:
weekend_ratio_ = weekend_ratio.reset_index(inplace=False)
Q_total_ = Q_total.reset_index(inplace=False)

In [68]:
plus_ratio = Q_total_.merge(weekend_ratio_[['고객번호', '주말_비율']], how='outer',left_on='CUSTNO',right_on = '고객번호')
plus_ratio

,CUSTNO,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,...,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부,label,D-Recency,Frequancy,Monetary,고객번호,주말_비율
0,1,0,2,4,3,0,1,0,0,2,...,1,0,1,1,0,1,238,19925355,1,0.21
1,1,4,1,1,1,0,4,0,0,1,...,4,0,4,1,0,5,259,21565315,1,0.21
2,1,1,4,3,4,0,4,0,0,0,...,3,4,2,1,1,1,356,34116312,1,0.21
3,1,4,1,4,1,0,4,0,0,0,...,1,1,2,1,1,3,386,32637747,1,0.21
4,1,2,2,1,1,0,1,0,0,0,...,2,1,1,1,0,4,312,17898365,1,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113191,19337,0,4,0,0,4,0,0,0,0,...,0,4,0,2,0,1,214,675189,19337,0.20
113192,19337,0,2,0,0,1,0,0,0,0,...,0,2,0,2,1,1,362,1368729,19337,0.20
113193,19337,0,1,0,0,0,0,0,4,0,...,0,2,0,2,0,1,310,1299880,19337,0.20
113194,19337,0,0,0,0,0,0,0,1,0,...,0,2,0,2,0,1,360,1194630,19337,0.20


In [69]:
plus_ratio.index = plus_ratio['CUSTNO']
plus_ratio.drop(['CUSTNO', '고객번호'], axis=1, inplace = True)

In [71]:
plus_ratio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113196 entries, 1 to 19337
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   가구인테리어/잡화_고가_증감여부    113196 non-null  int64  
 1   가구인테리어/잡화_저가_증감여부    113196 non-null  int64  
 2   가구인테리어/잡화_중가_증감여부    113196 non-null  int64  
 3   가전/레저취미/멀티샵_고가_증감여부  113196 non-null  int64  
 4   가전/레저취미/멀티샵_저가_증감여부  113196 non-null  int64  
 5   가전/레저취미/멀티샵_중가_증감여부  113196 non-null  int64  
 6   교육문화/아동_고가_증감여부      113196 non-null  int64  
 7   교육문화/아동_저가_증감여부      113196 non-null  int64  
 8   교육문화/아동_중가_증감여부      113196 non-null  int64  
 9   식품/생활_고가_증감여부        113196 non-null  int64  
 10  식품/생활_저가_증감여부        113196 non-null  int64  
 11  식품/생활_중가_증감여부        113196 non-null  int64  
 12  의류/이미용_고가_증감여부       113196 non-null  int64  
 13  의류/이미용_저가_증감여부       113196 non-null  int64  
 14  의류/이미용_중가_증감여부       113196 non-null  int64  
 15  침구/주방_고가_증감여부     

In [78]:
plus_ratio.fillna(0, inplace = True)

## +주말_비율 추가

In [79]:
get_model_train_eval(plus_ratio, model_li)

DecisionTreeClassifier()
정확도: 0.5503091872791519
RandomForestClassifier()
정확도: 0.6268992932862191
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.6221731448763251


# 고객DEMO 추가

In [80]:
df_sex_age = pd.read_csv('C:/cakd7/git/class/study/m5_데이터베이스/2차 플젝/생성자료/고객DEMO.txt', sep = ',', encoding = 'cp949')
df_sex_age

,고객번호,성별,연령대,거주지역
0,1,M,60세이상,60.0
1,2,M,60세이상,100.0
2,3,M,60세이상,33.0
3,4,F,60세이상,16.0
4,5,M,60세이상,100.0
...,...,...,...,...
19378,19379,F,20세~24세,69.0
19379,19380,F,20세~24세,39.0
19380,19381,F,40세~44세,460.0
19381,19382,F,45세~49세,460.0


In [83]:
plus_ratio_ = plus_ratio.reset_index(inplace=False)
plus_demo = plus_ratio_.merge(df_sex_age[['고객번호', '성별', '연령대']], how='left',left_on='CUSTNO',right_on = '고객번호')
plus_demo

,CUSTNO,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,...,침구/주방_중가_증감여부,total증감여부,label,D-Recency,Frequancy,Monetary,주말_비율,고객번호,성별,연령대
0,1,0,2,4,3,0,1,0,0,2,...,1,1,0,1,238,19925355,0.21,1,M,60세이상
1,1,4,1,1,1,0,4,0,0,1,...,4,1,0,5,259,21565315,0.21,1,M,60세이상
2,1,1,4,3,4,0,4,0,0,0,...,2,1,1,1,356,34116312,0.21,1,M,60세이상
3,1,4,1,4,1,0,4,0,0,0,...,2,1,1,3,386,32637747,0.21,1,M,60세이상
4,1,2,2,1,1,0,1,0,0,0,...,1,1,0,4,312,17898365,0.21,1,M,60세이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113191,19337,0,4,0,0,4,0,0,0,0,...,0,2,0,1,214,675189,0.20,19337,M,20세~24세
113192,19337,0,2,0,0,1,0,0,0,0,...,0,2,1,1,362,1368729,0.20,19337,M,20세~24세
113193,19337,0,1,0,0,0,0,0,4,0,...,0,2,0,1,310,1299880,0.20,19337,M,20세~24세
113194,19337,0,0,0,0,0,0,0,1,0,...,0,2,0,1,360,1194630,0.20,19337,M,20세~24세


In [85]:
plus_demo.index = plus_demo['CUSTNO']
plus_demo.drop(['CUSTNO', '고객번호'], axis=1, inplace = True)
plus_demo

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,...,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부,label,D-Recency,Frequancy,Monetary,주말_비율,성별,연령대
CUSTNO,,,,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,...,0,1,1,0,1,238,19925355,0.21,M,60세이상
1,4,1,1,1,0,4,0,0,1,1,...,0,4,1,0,5,259,21565315,0.21,M,60세이상
1,1,4,3,4,0,4,0,0,0,4,...,4,2,1,1,1,356,34116312,0.21,M,60세이상
1,4,1,4,1,0,4,0,0,0,2,...,1,2,1,1,3,386,32637747,0.21,M,60세이상
1,2,2,1,1,0,1,0,0,0,3,...,1,1,1,0,4,312,17898365,0.21,M,60세이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19337,0,4,0,0,4,0,0,0,0,4,...,4,0,2,0,1,214,675189,0.20,M,20세~24세
19337,0,2,0,0,1,0,0,0,0,4,...,2,0,2,1,1,362,1368729,0.20,M,20세~24세
19337,0,1,0,0,0,0,0,4,0,1,...,2,0,2,0,1,310,1299880,0.20,M,20세~24세


In [89]:
X_features_ohe = pd.get_dummies(plus_demo[['성별', '연령대']])
X_features_ohe

,성별_F,성별_M,연령대_19세이하,연령대_20세~24세,연령대_25세~29세,연령대_30세~34세,연령대_35세~39세,연령대_40세~44세,연령대_45세~49세,연령대_50세~54세,연령대_55세~59세,연령대_60세이상
CUSTNO,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
19337,0,1,0,1,0,0,0,0,0,0,0,0
19337,0,1,0,1,0,0,0,0,0,0,0,0
19337,0,1,0,1,0,0,0,0,0,0,0,0


In [91]:
x = plus_demo.drop(['label', '성별', '연령대'], axis = 1)
X_ohe = pd.concat([x, X_features_ohe], axis=1)
X_ohe

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,...,연령대_19세이하,연령대_20세~24세,연령대_25세~29세,연령대_30세~34세,연령대_35세~39세,연령대_40세~44세,연령대_45세~49세,연령대_50세~54세,연령대_55세~59세,연령대_60세이상
CUSTNO,,,,,,,,,,,,,,,,,,,,,
1,0,2,4,3,0,1,0,0,2,4,...,0,0,0,0,0,0,0,0,0,1
1,4,1,1,1,0,4,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,4,3,4,0,4,0,0,0,4,...,0,0,0,0,0,0,0,0,0,1
1,4,1,4,1,0,4,0,0,0,2,...,0,0,0,0,0,0,0,0,0,1
1,2,2,1,1,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19337,0,4,0,0,4,0,0,0,0,4,...,0,1,0,0,0,0,0,0,0,0
19337,0,2,0,0,1,0,0,0,0,4,...,0,1,0,0,0,0,0,0,0,0
19337,0,1,0,0,0,0,0,4,0,1,...,0,1,0,0,0,0,0,0,0,0


In [94]:
y = plus_demo['label']
y

CUSTNO
1        0
1        0
1        1
1        1
1        0
        ..
19337    0
19337    1
19337    0
19337    0
19337    1
Name: label, Length: 113196, dtype: int32

In [92]:
X_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113196 entries, 1 to 19337
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   가구인테리어/잡화_고가_증감여부    113196 non-null  int64  
 1   가구인테리어/잡화_저가_증감여부    113196 non-null  int64  
 2   가구인테리어/잡화_중가_증감여부    113196 non-null  int64  
 3   가전/레저취미/멀티샵_고가_증감여부  113196 non-null  int64  
 4   가전/레저취미/멀티샵_저가_증감여부  113196 non-null  int64  
 5   가전/레저취미/멀티샵_중가_증감여부  113196 non-null  int64  
 6   교육문화/아동_고가_증감여부      113196 non-null  int64  
 7   교육문화/아동_저가_증감여부      113196 non-null  int64  
 8   교육문화/아동_중가_증감여부      113196 non-null  int64  
 9   식품/생활_고가_증감여부        113196 non-null  int64  
 10  식품/생활_저가_증감여부        113196 non-null  int64  
 11  식품/생활_중가_증감여부        113196 non-null  int64  
 12  의류/이미용_고가_증감여부       113196 non-null  int64  
 13  의류/이미용_저가_증감여부       113196 non-null  int64  
 14  의류/이미용_중가_증감여부       113196 non-null  int64  
 15  침구/주방_고가_증감여부     

In [41]:
q_df[0]

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,4,1,1,1,0,4,0,0,1,1,4,4,3,0,3,4,0,4
2,0,3,1,3,0,1,0,0,1,1,0,2,3,0,4,3,0,4
3,0,4,0,0,4,0,0,0,0,2,2,4,0,0,1,0,4,0
4,0,3,1,3,1,3,0,0,0,1,1,1,2,4,2,4,1,4
6,3,1,1,0,1,0,0,0,0,3,3,2,4,2,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,2,0,0,0,0,0,0,1,1,1,2,0,0,0,0,0,0
19334,0,4,0,0,1,0,4,2,2,2,3,2,0,2,0,0,1,0
19335,0,4,0,0,1,1,1,2,3,2,1,2,0,1,1,0,3,1


In [95]:
X_features_ohe = pd.get_dummies(plus_demo[['성별', '연령대']])

def get_clf_eval(y_test,pred):
    accuracy = accuracy_score(y_test,pred)
    print(f'정확도: {accuracy}')
    
def get_model_train_eval_ohe(df, models):
    x = df.drop(['label'], axis = 1)
    X_ohe = pd.concat([x, X_features_ohe], axis=1)
    print(X_ohe)
    y = df['label']

    # 학습/테스트용 분리
    X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size = 0.2, random_state = 11)
    print(X_train)
    for model in models:
        print(model)
        model.fit(X_train, y_train)
        pred=model.predict(X_test)
        get_clf_eval(y_test,pred)

# 모델리스트
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, 
                         boost_from_average=False)
model_li = [dt_clf, rf_clf, lgbm_clf]

get_model_train_eval(plus_demo.drop(['성별', '연령대'], axis=1), model_li)

DecisionTreeClassifier()
정확도: 0.547482332155477
RandomForestClassifier()
정확도: 0.6260600706713781
LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)
정확도: 0.6221731448763251


In [104]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, accuracy_score,  confusion_matrix, confusion_matrix, f1_score, recall_score
from xgboost import XGBClassifier

# def objecdtive_func(search_space):
#     lgbm_clf = LGBMClassifier(n_estimators = 100,
#                                             num_leaves = int(search_space['num_leaves']),
#                                             max_depth = int(search_space['max_depth']),
#                                             min_child_samples = int(search_space['min_child_samples']),
#                                             subsample = search_space['subsample']
#                                             learning_rate = search_space['learning_rate'])

X_features_ohe = pd.get_dummies(plus_demo[['성별', '연령대']])
x = plus_demo.drop(['label', '성별', '연령대'], axis=1)
X_ohe = pd.concat([x, X_features_ohe], axis=1)

y = plus_demo['label']

# 학습/테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size = 0.2, random_state = 11)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=156)

lgbm_wrapper = LGBMClassifier(n_estimators = 400, leargning_rate = 0.05)


# LGBM 조기 중단 수행 가능
## early_stopping_rounds: 50번 수행하는데 계산이 안되면(logloss 손실이 낮아지는) 멈추기
## eval_metric = 'logloss': 오류함수의 평가 성능지표, 신경망의 성능이 얼마나 나쁜지를 나타내는 지표
## verbose = True: 과정 보여줌
evals = [(X_tr, y_tr), (X_val, y_val)]
lgbm_wrapper.fit(X_tr, y_tr, 
                 early_stopping_rounds = 50, 
                 eval_metric = 'logloss',
                 eval_set = evals, 
                 verbose = True)
# 예측
preds = lgbm_wrapper.predict(X_test)

pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]

def get_clf_eval(y_test , pred, pred_proba):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f},  AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
    
get_clf_eval(y_test, preds, pred_proba)
    
#     lgbm_clf = LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)


#     kfold = KFold(n_splits = 3)

#     roc_auc_list = []
#     for train_index, val_index in kfold.split(X_train):

#         X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
#         X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

#         lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric = "auc", eval_set = [(X_tr, y_tr), (X_val, y_val)])

#         score = roc_auc_score(y_val, lgbm_clf.predict_proba(X_val)[:, 1])
#         roc_auc_list.append(score)

#     return -1*np.mean(roc_auc_list)

[LightGBM] [Warning] Unknown parameter: leargning_rate
[1]	training's binary_logloss: 0.686552	valid_1's binary_logloss: 0.686927
[2]	training's binary_logloss: 0.681194	valid_1's binary_logloss: 0.681929
[3]	training's binary_logloss: 0.676534	valid_1's binary_logloss: 0.677549
[4]	training's binary_logloss: 0.67257	valid_1's binary_logloss: 0.674123
[5]	training's binary_logloss: 0.669075	valid_1's binary_logloss: 0.671098
[6]	training's binary_logloss: 0.666011	valid_1's binary_logloss: 0.668347
[7]	training's binary_logloss: 0.663449	valid_1's binary_logloss: 0.666152
[8]	training's binary_logloss: 0.660909	valid_1's binary_logloss: 0.663928
[9]	training's binary_logloss: 0.658583	valid_1's binary_logloss: 0.662061
[10]	training's binary_logloss: 0.656482	valid_1's binary_logloss: 0.660208
[11]	training's binary_logloss: 0.65458	valid_1's binary_logloss: 0.658419
[12]	training's binary_logloss: 0.652916	valid_1's binary_logloss: 0.657022
[13]	training's binary_logloss: 0.651385	val

[122]	training's binary_logloss: 0.605712	valid_1's binary_logloss: 0.635978
[123]	training's binary_logloss: 0.605559	valid_1's binary_logloss: 0.636001
[124]	training's binary_logloss: 0.605332	valid_1's binary_logloss: 0.636002
[125]	training's binary_logloss: 0.605129	valid_1's binary_logloss: 0.636074
[126]	training's binary_logloss: 0.604894	valid_1's binary_logloss: 0.636138
[127]	training's binary_logloss: 0.604683	valid_1's binary_logloss: 0.636163
[128]	training's binary_logloss: 0.604555	valid_1's binary_logloss: 0.636166
[129]	training's binary_logloss: 0.604346	valid_1's binary_logloss: 0.636231
[130]	training's binary_logloss: 0.604107	valid_1's binary_logloss: 0.636155
[131]	training's binary_logloss: 0.603907	valid_1's binary_logloss: 0.636166
[132]	training's binary_logloss: 0.603723	valid_1's binary_logloss: 0.636172
[133]	training's binary_logloss: 0.603507	valid_1's binary_logloss: 0.636201
[134]	training's binary_logloss: 0.603373	valid_1's binary_logloss: 0.636177

In [99]:
!pip install hyperopt

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=8d3c760eab4ef158b58c8451db03d45b2537604bd8677654cba4b4885d2bc236
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\8e\70\28\3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future


In [102]:
from hyperopt import fmin, tpe, Trials, hp
lgbm_search_space = {'num_leaves': hp.quniform('num_leaves', 32, 64, 1),
                    'max_depth': hp.quniform('max_depth', 100, 160, 1),
                    'min_child_samples': hp.quniform('min_child_samples', 60, 100, 1),
                     'subsample': hp.uniform('subsample', 0.7,1),
                     'learning_rate': hp.uniform('learning_rate',0.01,0.2)}

In [100]:
from hyperopt import fmin, tpe, Trials, hp

trials = Trials()
best = fmin(fn = objective_func, space = lgbm_search_space, algo = tpe.suggest, max_evals = 50, trials = trials, rstate = np.random.default_rng(seed = 30))
print('best:', best)

NameError: name 'objective_func' is not defined

In [ ]:
get_model_train_eval(plus_ratio, model_li)

In [51]:
total_2

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,...,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부,total증감여부,label,D-Recency,Frequancy,Monetary
CUSTNO,,,,,,,,,,,,,,,,,,,,,
1,4,1,1,1,0,4,0,0,1,1,...,0,3,4,0,4,1,0,5,259,21565315
2,0,3,1,3,0,1,0,0,1,1,...,0,4,3,0,4,2,1,4,294,25040940
3,0,4,0,0,4,0,0,0,0,2,...,0,1,0,4,0,2,1,1,455,1196621
4,0,3,1,3,1,3,0,0,0,1,...,4,2,4,1,4,2,1,1,262,6166873
6,3,1,1,0,1,0,0,0,0,3,...,2,2,1,2,0,2,1,2,370,6278557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,2,0,0,0,0,0,0,1,1,...,0,0,0,0,0,4,1,3,260,1492198
19334,0,4,0,0,1,0,4,2,2,2,...,2,0,0,1,0,2,0,1,992,5281545
19335,0,4,0,0,1,1,1,2,3,2,...,1,1,0,3,1,3,1,2,569,5103227


In [ ]:
total_Q = pd.concat([])

In [39]:
total.columns[1:]

Index(['2분기 대비3분기 증감여부', '3분기 대비4분기 증감여부', '4분기 대비5분기 증감여부', '5분기 대비6분기 증감여부',
       '6분기 대비7분기 증감여부', '7분기 대비8분기 증감여부'],
      dtype='object')

In [47]:
for i in total.columns[1:]:
    print(total[i])

CUSTNO
1        3
2        4
3        3
4        2
6        3
        ..
19333    1
19334    2
19335    2
19336    3
19337    4
Name: 2분기 대비3분기 증감여부, Length: 18866, dtype: int64
CUSTNO
1        3
2        1
3        2
4        1
6        2
        ..
19333    2
19334    3
19335    2
19336    3
19337    3
Name: 3분기 대비4분기 증감여부, Length: 18866, dtype: int64
CUSTNO
1        1
2        4
3        2
4        3
6        3
        ..
19333    4
19334    1
19335    3
19336    1
19337    2
Name: 4분기 대비5분기 증감여부, Length: 18866, dtype: int64
CUSTNO
1        2
2        1
3        3
4        3
6        2
        ..
19333    2
19334    1
19335    3
19336    3
19337    3
Name: 5분기 대비6분기 증감여부, Length: 18866, dtype: int64
CUSTNO
1        3
2        4
3        3
4        3
6        3
        ..
19333    1
19334    2
19335    1
19336    4
19337    4
Name: 6분기 대비7분기 증감여부, Length: 18866, dtype: int64
CUSTNO
1        1
2        2
3        2
4        2
6        2
        ..
19333    4
19334    2
19335    3
1933

In [40]:
q_df = [df2, df3, df4, df5, df6]

In [44]:
num = 3

rfm+str(num)

NameError: name 'rfm' is not defined

## 5분기 대비6분기

In [8]:
df6 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q6_q5_df.csv', index_col = 0)
df6.head()

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,2,2,1,1,0,1,0,0,0,3,3,1,3,0,3,2,1,1
2,1,3,1,2,1,0,0,0,1,1,4,1,2,4,3,1,0,4
3,0,4,0,0,0,0,0,0,0,1,3,3,0,0,0,0,0,0
4,0,4,4,0,0,3,0,4,0,4,4,2,4,4,2,1,0,0
6,4,3,4,0,0,0,0,0,0,2,1,1,3,3,4,1,1,1


## 6분기 대비 7분기

In [41]:
q_df[0]

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,4,1,1,1,0,4,0,0,1,1,4,4,3,0,3,4,0,4
2,0,3,1,3,0,1,0,0,1,1,0,2,3,0,4,3,0,4
3,0,4,0,0,4,0,0,0,0,2,2,4,0,0,1,0,4,0
4,0,3,1,3,1,3,0,0,0,1,1,1,2,4,2,4,1,4
6,3,1,1,0,1,0,0,0,0,3,3,2,4,2,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,2,0,0,0,0,0,0,1,1,1,2,0,0,0,0,0,0
19334,0,4,0,0,1,0,4,2,2,2,3,2,0,2,0,0,1,0
19335,0,4,0,0,1,1,1,2,3,2,1,2,0,1,1,0,3,1


In [9]:
df7 = pd.read_csv('C:/cakd7/git/2차 프로젝트/second_projects/생성데이터/q7_q6_df.csv', index_col = 0)
df7.head()

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,1,1,4,4,0,3,0,0,0,1,1,1,1,0,3,3,0,0
2,0,3,0,4,0,0,0,0,4,4,1,2,2,1,4,3,0,1
3,0,3,0,0,4,0,0,0,0,4,1,4,0,0,0,0,0,0
4,0,2,4,0,4,4,0,1,0,1,1,2,1,4,3,4,0,0
6,3,1,4,0,0,0,0,4,0,4,4,4,2,3,3,0,4,4


In [10]:
df_total = df7 - df6
df_total

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,-1,-1,3,3,0,2,0,0,0,-2,-2,0,-2,0,0,1,-1,-1
2,-1,0,-1,2,-1,0,0,0,3,3,-3,1,0,-3,1,2,0,-3
3,0,-1,0,0,4,0,0,0,0,3,-2,1,0,0,0,0,0,0
4,0,-2,0,0,4,1,0,-3,0,-3,-3,0,-3,0,1,3,0,0
6,-1,-2,0,0,0,0,0,4,0,2,3,3,-1,0,-1,-1,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0,0,0,0,0,0,0,0,0,-3,1,0,0,0,0,0,0,0
19334,0,-1,0,0,0,0,3,-2,3,-2,-1,0,0,-2,4,0,0,0
19335,0,-2,-3,0,0,-3,-1,0,0,-3,2,0,-3,0,-1,0,-2,-1
